In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is the PRIMARY learning material!")

# Day 1: Tensor Core Basics - The Matrix Multiplication Accelerators

> **"Why use one LEGO brick at a time when you can snap together entire 4×4 blocks in a single motion?"** That's the power of Tensor Cores!

---

## 🎯 Learning Objectives

By the end of this session, you will:
1. **Understand** what Tensor Cores are and why they deliver 8-16× speedups
2. **Compare** CUDA Core vs Tensor Core execution models
3. **Identify** supported precision formats (FP16, TF32, BF16, INT8)
4. **Recognize** when to leverage Tensor Cores in your applications

---

## 🎨 Concept Card: LEGO Brick Assembly

```
┌─────────────────────────────────────────────────────────────────┐
│  🧱 TENSOR CORES = LEGO BLOCK FACTORY                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  CUDA Core (One Brick at a Time):                               │
│  ┌───┐ + ┌───┐ = ┌───┐                                          │
│  │ × │   │ + │   │ = │   One FMA, one result                    │
│  └───┘   └───┘   └───┘                                          │
│                                                                 │
│  Tensor Core (Entire 4×4 Block at Once):                        │
│  ┌───┬───┬───┬───┐     ┌───┬───┬───┬───┐     ┌───┬───┬───┬───┐  │
│  │ A │ A │ A │ A │     │ B │ B │ B │ B │     │ C │ C │ C │ C │  │
│  ├───┼───┼───┼───┤  ×  ├───┼───┼───┼───┤  +  ├───┼───┼───┼───┤  │
│  │ A │ A │ A │ A │     │ B │ B │ B │ B │     │ C │ C │ C │ C │  │
│  ├───┼───┼───┼───┤     ├───┼───┼───┼───┤     ├───┼───┼───┼───┤  │
│  │ A │ A │ A │ A │     │ B │ B │ B │ B │     │ C │ C │ C │ C │  │
│  ├───┼───┼───┼───┤     ├───┼───┼───┼───┤     ├───┼───┼───┼───┤  │
│  │ A │ A │ A │ A │     │ B │ B │ B │ B │     │ C │ C │ C │ C │  │
│  └───┴───┴───┴───┘     └───┴───┴───┴───┘     └───┴───┴───┴───┘  │
│         │                     │                     │           │
│         └─────────────────────┴─────────────────────┘           │
│                               ▼                                 │
│                    64 FMAs in ONE operation!                    │
│                                                                 │
│  💡 Key Insight:                                                │
│  • CUDA Core:   1 operation per cycle                           │
│  • Tensor Core: 64 operations per cycle (128 FLOPs!)            │
│  • Like assembling a 4×4 LEGO section vs individual bricks      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: What Are Tensor Cores?

### CUDA Cores vs Tensor Cores

```
CUDA Core (Traditional):
┌─────────────────────────────────────┐
│  One FMA per cycle                  │
│  D = A × B + C                      │
│  (scalar operation)                 │
└─────────────────────────────────────┘

Tensor Core:
┌─────────────────────────────────────┐
│  4×4×4 Matrix FMA per cycle         │
│  D[4×4] = A[4×4] × B[4×4] + C[4×4]  │
│  (64 FMAs = 128 operations!)        │
└─────────────────────────────────────┘
```

### Tensor Core Pipeline

```
Warp (32 threads) cooperates on:

    A (16×16)      B (16×16)      C (16×16)
    ┌──────┐       ┌──────┐       ┌──────┐
    │ FP16 │   ×   │ FP16 │   +   │ FP32 │
    └──────┘       └──────┘       └──────┘
         │              │              │
         └──────────────┼──────────────┘
                        ▼
                   D (16×16)
                   ┌──────┐
                   │ FP32 │
                   └──────┘

One WMMA operation = 16×16×16 matrix multiply
```

### Performance Comparison

| GPU | FP32 CUDA Cores | FP16 Tensor Cores |
|-----|-----------------|-------------------|
| V100 | 15.7 TFLOPS | 125 TFLOPS |
| A100 | 19.5 TFLOPS | 312 TFLOPS |
| H100 | 67 TFLOPS | 989 TFLOPS |

**8-16× speedup for matrix operations!**

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile tensor_core_check.cu
// tensor_core_check.cu - Check Tensor Core support
#include <stdio.h>
#include <cuda_runtime.h>

void checkTensorCoreSupport() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    printf("=== Tensor Core Support Check ===\n\n");
    
    for (int i = 0; i < deviceCount; i++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, i);
        
        printf("Device %d: %s\n", i, prop.name);
        printf("  Compute Capability: %d.%d\n", prop.major, prop.minor);
        
        // Tensor Cores require SM 7.0+
        int smVersion = prop.major * 10 + prop.minor;
        
        if (smVersion >= 70) {
            printf("  Tensor Cores: ✅ SUPPORTED\n");
            
            // Determine supported data types
            printf("  Supported Types:\n");
            printf("    - FP16 (half precision)\n");
            
            if (smVersion >= 75) {
                printf("    - INT8, INT4\n");
            }
            if (smVersion >= 80) {
                printf("    - BF16 (bfloat16)\n");
                printf("    - TF32 (TensorFloat-32)\n");
            }
            if (smVersion >= 90) {
                printf("    - FP8\n");
            }
        } else {
            printf("  Tensor Cores: ❌ NOT SUPPORTED (need SM 7.0+)\n");
        }
        
        printf("\n");
    }
}

int main() {
    checkTensorCoreSupport();
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o tensor_core_check tensor_core_check.cu
!./tensor_core_check

---

## Part 2: WMMA (Warp Matrix Multiply-Accumulate)

### WMMA Fragment Types

```
WMMA API uses "fragments" - distributed matrix pieces:

fragment<matrix_a, M, N, K, half, row_major> a_frag;
fragment<matrix_b, M, N, K, half, col_major> b_frag;
fragment<accumulator, M, N, K, float> c_frag;

Supported shapes (M×N×K):
  - 16×16×16 (most common)
  - 32×8×16
  - 8×32×16
```

### WMMA Operations

```cpp
// 1. Load matrices into fragments
load_matrix_sync(a_frag, A_ptr, lda);
load_matrix_sync(b_frag, B_ptr, ldb);
fill_fragment(c_frag, 0.0f);

// 2. Matrix multiply-accumulate
mma_sync(c_frag, a_frag, b_frag, c_frag);

// 3. Store result
store_matrix_sync(C_ptr, c_frag, ldc, mem_row_major);
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile wmma_basic.cu
// wmma_basic.cu - Basic WMMA example
#include <stdio.h>
#include <cuda_runtime.h>
#include <mma.h>
#include <cuda_fp16.h>

using namespace nvcuda;

// WMMA dimensions
const int WMMA_M = 16;
const int WMMA_N = 16;
const int WMMA_K = 16;

__global__ void wmma_matmul_simple(
    half* A, half* B, float* C,
    int M, int N, int K
) {
    // Each warp computes one 16x16 tile of C
    int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / 32;
    int warpN = blockIdx.y;
    
    // Bounds check
    if (warpM * WMMA_M >= M || warpN * WMMA_N >= N) return;
    
    // Declare fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::col_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> c_frag;
    
    // Initialize accumulator to zero
    wmma::fill_fragment(c_frag, 0.0f);
    
    // Loop over K dimension in tiles of WMMA_K
    for (int k = 0; k < K; k += WMMA_K) {
        int aRow = warpM * WMMA_M;
        int aCol = k;
        int bRow = k;
        int bCol = warpN * WMMA_N;
        
        // Load A and B tiles
        wmma::load_matrix_sync(a_frag, A + aRow * K + aCol, K);
        wmma::load_matrix_sync(b_frag, B + bRow * N + bCol, N);
        
        // Perform matrix multiply-accumulate
        wmma::mma_sync(c_frag, a_frag, b_frag, c_frag);
    }
    
    // Store result
    int cRow = warpM * WMMA_M;
    int cCol = warpN * WMMA_N;
    wmma::store_matrix_sync(C + cRow * N + cCol, c_frag, N, wmma::mem_row_major);
}

// Helper to convert float to half
__global__ void float2half(float* in, half* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        out[idx] = __float2half(in[idx]);
    }
}

int main() {
    const int M = 256, N = 256, K = 256;
    
    // Allocate host memory
    float *h_A = new float[M * K];
    float *h_B = new float[K * N];
    float *h_C = new float[M * N];
    
    // Initialize with simple values
    for (int i = 0; i < M * K; i++) h_A[i] = 1.0f;
    for (int i = 0; i < K * N; i++) h_B[i] = 1.0f;
    
    // Allocate device memory
    float *d_A_float, *d_B_float, *d_C;
    half *d_A, *d_B;
    
    cudaMalloc(&d_A_float, M * K * sizeof(float));
    cudaMalloc(&d_B_float, K * N * sizeof(float));
    cudaMalloc(&d_A, M * K * sizeof(half));
    cudaMalloc(&d_B, K * N * sizeof(half));
    cudaMalloc(&d_C, M * N * sizeof(float));
    
    // Copy and convert to half precision
    cudaMemcpy(d_A_float, h_A, M * K * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B_float, h_B, K * N * sizeof(float), cudaMemcpyHostToDevice);
    
    float2half<<<(M*K+255)/256, 256>>>(d_A_float, d_A, M*K);
    float2half<<<(K*N+255)/256, 256>>>(d_B_float, d_B, K*N);
    
    // Launch WMMA kernel
    // Each block has 4 warps (128 threads), each warp handles 16x16 tile
    dim3 block(128);
    dim3 grid((M / WMMA_M + 3) / 4, N / WMMA_N);
    
    // Warmup
    wmma_matmul_simple<<<grid, block>>>(d_A, d_B, d_C, M, N, K);
    cudaDeviceSynchronize();
    
    // Benchmark
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        wmma_matmul_simple<<<grid, block>>>(d_A, d_B, d_C, M, N, K);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    // Copy result back
    cudaMemcpy(h_C, d_C, M * N * sizeof(float), cudaMemcpyDeviceToHost);
    
    // Verify (each element should be K = 256)
    printf("WMMA Matrix Multiply Results:\n");
    printf("  Matrix size: %d x %d x %d\n", M, N, K);
    printf("  C[0,0] = %.1f (expected: %d)\n", h_C[0], K);
    printf("  C[M-1,N-1] = %.1f (expected: %d)\n", h_C[M*N-1], K);
    printf("  Time: %.3f ms (100 iterations)\n", ms);
    printf("  Avg time: %.3f ms\n", ms / 100);
    
    // Calculate TFLOPS
    double flops = 2.0 * M * N * K;  // multiply-add = 2 ops
    double tflops = (flops * 100) / (ms * 1e9);
    printf("  Performance: %.2f TFLOPS\n", tflops);
    
    // Cleanup
    delete[] h_A; delete[] h_B; delete[] h_C;
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    cudaFree(d_A_float); cudaFree(d_B_float);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o wmma_basic wmma_basic.cu
!./wmma_basic

---

## Part 3: Data Types for Tensor Cores

### Precision Hierarchy

```
FP32 (32-bit float)
├── Full precision, traditional CUDA
├── 1 sign + 8 exp + 23 mantissa
└── Range: ±3.4 × 10^38

TF32 (TensorFloat-32) - Ampere+
├── 19-bit format (stored as FP32)
├── 1 sign + 8 exp + 10 mantissa  
├── Same range as FP32, less precision
└── Automatic in cuBLAS!

FP16 (16-bit half)
├── 1 sign + 5 exp + 10 mantissa
├── Range: ±65,504
└── 2x memory bandwidth

BF16 (bfloat16) - Ampere+
├── 1 sign + 8 exp + 7 mantissa
├── Same range as FP32
└── Truncated FP32, easy conversion
```

### When to Use Each

| Type | Best For | Gotchas |
|------|----------|--------|
| FP32 | Baseline, accumulation | Slow on Tensor Cores |
| TF32 | Drop-in FP32 speedup | Ampere+ only |
| FP16 | Training, inference | Narrow range, overflow |
| BF16 | Training stability | Ampere+ only |

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile data_types.cu
// data_types.cu - Understanding FP16, BF16, TF32
#include <stdio.h>
#include <cuda_runtime.h>
#include <cuda_fp16.h>

// Analyze floating point representations
void analyzeFloat(float val) {
    printf("\nAnalyzing: %f\n", val);
    
    // FP32 bits
    unsigned int* fp32_bits = (unsigned int*)&val;
    printf("  FP32: 0x%08X\n", *fp32_bits);
    printf("    Sign: %d, Exp: %d, Mantissa: 0x%06X\n",
           (*fp32_bits >> 31) & 1,
           ((*fp32_bits >> 23) & 0xFF) - 127,
           *fp32_bits & 0x7FFFFF);
    
    // Convert to FP16
    half h_val = __float2half(val);
    unsigned short* fp16_bits = (unsigned short*)&h_val;
    printf("  FP16: 0x%04X\n", *fp16_bits);
    printf("    Sign: %d, Exp: %d, Mantissa: 0x%03X\n",
           (*fp16_bits >> 15) & 1,
           ((*fp16_bits >> 10) & 0x1F) - 15,
           *fp16_bits & 0x3FF);
    
    // Round-trip precision loss
    float back = __half2float(h_val);
    printf("  Round-trip: %f -> FP16 -> %f (error: %e)\n",
           val, back, fabs(val - back));
}

__global__ void fp16_overflow_demo(half* out) {
    // FP16 max is ~65504
    half a = __float2half(60000.0f);
    half b = __float2half(60000.0f);
    half c = __hadd(a, b);  // Will overflow!
    out[0] = c;
}

int main() {
    printf("=== Floating Point Data Types ===\n");
    
    analyzeFloat(1.0f);
    analyzeFloat(3.14159f);
    analyzeFloat(0.00001f);
    analyzeFloat(65504.0f);  // FP16 max
    
    // Demonstrate FP16 overflow
    printf("\n=== FP16 Overflow Demo ===\n");
    half *d_out, h_out;
    cudaMalloc(&d_out, sizeof(half));
    fp16_overflow_demo<<<1, 1>>>(d_out);
    cudaMemcpy(&h_out, d_out, sizeof(half), cudaMemcpyDeviceToHost);
    printf("60000 + 60000 in FP16 = %f (overflow!)\n", __half2float(h_out));
    
    // Show FP16 range limits
    printf("\n=== FP16 Limits ===\n");
    printf("  Max positive: 65504\n");
    printf("  Min positive (normalized): ~6.1e-5\n");
    printf("  Min positive (denorm): ~5.96e-8\n");
    
    cudaFree(d_out);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o data_types data_types.cu
!./data_types

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python - Understanding precision differences
import numpy as np

def compare_precisions():
    """Compare FP32 vs FP16 precision."""
    print("Precision Comparison")
    print("=" * 50)
    
    # Test values
    values = [1.0, 3.14159, 0.00001, 65504.0, 70000.0]
    
    for v in values:
        fp32 = np.float32(v)
        fp16 = np.float16(v)
        error = abs(float(fp32) - float(fp16))
        
        print(f"\nValue: {v}")
        print(f"  FP32: {fp32}")
        print(f"  FP16: {fp16}")
        print(f"  Error: {error}")
        
        if np.isinf(fp16):
            print("  ⚠️ OVERFLOW in FP16!")

compare_precisions()

---

## Part 4: Tensor Core Requirements

### Memory Alignment

```
Tensor Cores require aligned memory:

  ✅ Aligned (works):    address % 256 == 0
  ❌ Unaligned (slow):   random address

Leading dimension constraints:
  - Must be multiple of 8 for FP16
  - Must be multiple of 4 for TF32
```

### Matrix Shape Requirements

```
M, N, K should be multiples of:
  - 16 for FP16 WMMA
  - 8 for TF32

For non-conforming sizes:
  1. Pad matrices to required multiple
  2. Or handle edge tiles specially
```

In [ ]:
def tensor_core_requirements():
    """Print Tensor Core requirements."""
    print("Tensor Core Requirements Checklist")
    print("=" * 50)
    print()
    print("1. GPU REQUIREMENTS")
    print("   □ Compute Capability 7.0+ (Volta, Turing, Ampere)")
    print("   □ Tensor Cores present")
    print()
    print("2. MATRIX DIMENSIONS")
    print("   □ M, N, K multiples of 16 (FP16 WMMA)")
    print("   □ M, N, K multiples of 8 (TF32)")
    print("   □ Or handle edge tiles explicitly")
    print()
    print("3. MEMORY ALIGNMENT")
    print("   □ Matrix pointers 256-byte aligned")
    print("   □ Leading dimensions multiple of 8 (FP16)")
    print("   □ Use cudaMalloc (automatically aligned)")
    print()
    print("4. DATA TYPES")
    print("   □ Input: FP16 (half), BF16, TF32, or INT8")
    print("   □ Accumulator: FP32 or FP16")
    print()
    print("5. THREAD ORGANIZATION")
    print("   □ Warp-level operations (32 threads)")
    print("   □ All threads in warp must participate")

tensor_core_requirements()

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile tensor_core_exercises.cu
#include <cuda_runtime.h>
#include <cuda_fp16.h>
#include <mma.h>
#include <stdio.h>
#include <stdlib.h>

using namespace nvcuda;

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
            exit(1); \
        } \
    } while(0)

// ============================================================
// Exercise 1: Check GPU Tensor Core Support
// ============================================================

void exercise1_checkTensorCoreSupport() {
    printf("=== Exercise 1: Check Tensor Core Support ===\n");
    
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    for (int dev = 0; dev < deviceCount; dev++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, dev);
        
        int major = prop.major;
        int minor = prop.minor;
        
        printf("Device %d: %s\n", dev, prop.name);
        printf("  Compute Capability: %d.%d\n", major, minor);
        
        // Tensor Core support check
        bool hasTensorCores = false;
        const char* tcType = "None";
        
        if (major >= 7) {
            hasTensorCores = true;
            if (major == 7 && minor == 0) tcType = "V100 (FP16)";
            else if (major == 7 && minor == 5) tcType = "Turing (FP16, INT8)";
            else if (major == 8 && minor == 0) tcType = "A100 (FP16, TF32, BF16, INT8)";
            else if (major == 8 && minor == 6) tcType = "RTX 30xx (FP16, TF32, BF16)";
            else if (major >= 9) tcType = "Latest (FP16, TF32, BF16, FP8)";
            else tcType = "Generic Tensor Cores";
        }
        
        printf("  Tensor Cores: %s (%s)\n", hasTensorCores ? "YES" : "NO", tcType);
        printf("  Max Threads/Block: %d\n", prop.maxThreadsPerBlock);
        printf("  Shared Memory/Block: %zu KB\n\n", prop.sharedMemPerBlock / 1024);
    }
}

// ============================================================
// Exercise 2: Precision Analysis (FP32 vs FP16)
// ============================================================

__global__ void matmulFP32(const float* A, const float* B, float* C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; k++) {
            sum += A[row * K + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

__global__ void matmulFP16(const half* A, const half* B, float* C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; k++) {
            sum += __half2float(A[row * K + k]) * __half2float(B[k * N + col]);
        }
        C[row * N + col] = sum;
    }
}

void exercise2_precisionAnalysis() {
    printf("=== Exercise 2: Precision Analysis ===\n");
    
    const int M = 256, N = 256, K = 256;
    
    // Allocate host memory
    float* h_A = (float*)malloc(M * K * sizeof(float));
    float* h_B = (float*)malloc(K * N * sizeof(float));
    float* h_C_fp32 = (float*)malloc(M * N * sizeof(float));
    float* h_C_fp16 = (float*)malloc(M * N * sizeof(float));
    
    // Initialize with random values
    for (int i = 0; i < M * K; i++) h_A[i] = (float)(rand() % 100) / 100.0f;
    for (int i = 0; i < K * N; i++) h_B[i] = (float)(rand() % 100) / 100.0f;
    
    // Device memory
    float *d_A_fp32, *d_B_fp32, *d_C_fp32;
    half *d_A_fp16, *d_B_fp16;
    float *d_C_fp16;
    
    CHECK_CUDA(cudaMalloc(&d_A_fp32, M * K * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_B_fp32, K * N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_C_fp32, M * N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_A_fp16, M * K * sizeof(half)));
    CHECK_CUDA(cudaMalloc(&d_B_fp16, K * N * sizeof(half)));
    CHECK_CUDA(cudaMalloc(&d_C_fp16, M * N * sizeof(float)));
    
    CHECK_CUDA(cudaMemcpy(d_A_fp32, h_A, M * K * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_B_fp32, h_B, K * N * sizeof(float), cudaMemcpyHostToDevice));
    
    // Convert to FP16
    half* h_A_fp16 = (half*)malloc(M * K * sizeof(half));
    half* h_B_fp16 = (half*)malloc(K * N * sizeof(half));
    for (int i = 0; i < M * K; i++) h_A_fp16[i] = __float2half(h_A[i]);
    for (int i = 0; i < K * N; i++) h_B_fp16[i] = __float2half(h_B[i]);
    CHECK_CUDA(cudaMemcpy(d_A_fp16, h_A_fp16, M * K * sizeof(half), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_B_fp16, h_B_fp16, K * N * sizeof(half), cudaMemcpyHostToDevice));
    
    dim3 block(16, 16);
    dim3 grid((N + 15) / 16, (M + 15) / 16);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // FP32 benchmark
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        matmulFP32<<<grid, block>>>(d_A_fp32, d_B_fp32, d_C_fp32, M, N, K);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float fp32Ms;
    cudaEventElapsedTime(&fp32Ms, start, stop);
    
    // FP16 benchmark
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        matmulFP16<<<grid, block>>>(d_A_fp16, d_B_fp16, d_C_fp16, M, N, K);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float fp16Ms;
    cudaEventElapsedTime(&fp16Ms, start, stop);
    
    // Copy results back
    CHECK_CUDA(cudaMemcpy(h_C_fp32, d_C_fp32, M * N * sizeof(float), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(h_C_fp16, d_C_fp16, M * N * sizeof(float), cudaMemcpyDeviceToHost));
    
    // Compute error metrics
    float maxError = 0.0f;
    float meanAbsError = 0.0f;
    for (int i = 0; i < M * N; i++) {
        float error = fabsf(h_C_fp32[i] - h_C_fp16[i]);
        maxError = fmaxf(maxError, error);
        meanAbsError += error;
    }
    meanAbsError /= (M * N);
    
    printf("Matrix size: %d x %d x %d\n", M, N, K);
    printf("FP32 time: %.2f ms\n", fp32Ms);
    printf("FP16 time: %.2f ms\n", fp16Ms);
    printf("Speedup: %.2fx\n", fp32Ms / fp16Ms);
    printf("Max element-wise error: %.6f\n", maxError);
    printf("Mean absolute error: %.6f\n\n", meanAbsError);
    
    // Cleanup
    free(h_A); free(h_B); free(h_C_fp32); free(h_C_fp16);
    free(h_A_fp16); free(h_B_fp16);
    cudaFree(d_A_fp32); cudaFree(d_B_fp32); cudaFree(d_C_fp32);
    cudaFree(d_A_fp16); cudaFree(d_B_fp16); cudaFree(d_C_fp16);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

// ============================================================
// Exercise 3: Basic WMMA Matrix Multiply
// ============================================================

const int WMMA_M = 16;
const int WMMA_N = 16;
const int WMMA_K = 16;

__global__ void wmmaMatmul(const half* A, const half* B, float* C, int M, int N, int K) {
    // Calculate warp position
    int warpM = (blockIdx.y * blockDim.y + threadIdx.y) / 32;
    int warpN = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Skip if out of bounds
    if (warpM * WMMA_M >= M || warpN * WMMA_N >= N) return;
    
    // Declare fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> c_frag;
    
    // Initialize accumulator
    wmma::fill_fragment(c_frag, 0.0f);
    
    // Loop over K dimension
    for (int k = 0; k < K; k += WMMA_K) {
        int aRow = warpM * WMMA_M;
        int aCol = k;
        int bRow = k;
        int bCol = warpN * WMMA_N;
        
        // Load fragments
        wmma::load_matrix_sync(a_frag, A + aRow * K + aCol, K);
        wmma::load_matrix_sync(b_frag, B + bRow * N + bCol, N);
        
        // Perform WMMA
        wmma::mma_sync(c_frag, a_frag, b_frag, c_frag);
    }
    
    // Store result
    int cRow = warpM * WMMA_M;
    int cCol = warpN * WMMA_N;
    wmma::store_matrix_sync(C + cRow * N + cCol, c_frag, N, wmma::mem_row_major);
}

void exercise3_basicWMMA() {
    printf("=== Exercise 3: Basic WMMA Matrix Multiply ===\n");
    
    // Check compute capability
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    if (prop.major < 7) {
        printf("WMMA requires compute capability 7.0+. Skipping.\n\n");
        return;
    }
    
    const int M = 256, N = 256, K = 256;
    
    half* h_A = (half*)malloc(M * K * sizeof(half));
    half* h_B = (half*)malloc(K * N * sizeof(half));
    float* h_C = (float*)malloc(M * N * sizeof(float));
    
    for (int i = 0; i < M * K; i++) h_A[i] = __float2half((float)(rand() % 10) / 10.0f);
    for (int i = 0; i < K * N; i++) h_B[i] = __float2half((float)(rand() % 10) / 10.0f);
    
    half *d_A, *d_B;
    float *d_C;
    CHECK_CUDA(cudaMalloc(&d_A, M * K * sizeof(half)));
    CHECK_CUDA(cudaMalloc(&d_B, K * N * sizeof(half)));
    CHECK_CUDA(cudaMalloc(&d_C, M * N * sizeof(float)));
    
    CHECK_CUDA(cudaMemcpy(d_A, h_A, M * K * sizeof(half), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_B, h_B, K * N * sizeof(half), cudaMemcpyHostToDevice));
    
    // Launch with one warp per WMMA tile
    dim3 block(N / WMMA_N, 32);  // 32 threads = 1 warp per column
    dim3 grid(1, M / WMMA_M);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        wmmaMatmul<<<grid, block>>>(d_A, d_B, d_C, M, N, K);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    printf("WMMA Matrix Multiply (%dx%dx%d): %.2f ms (100 iters)\n", M, N, K, ms);
    printf("Performance: %.2f GFLOPS\n\n", 
           (2.0 * M * N * K * 100) / (ms * 1e6));
    
    free(h_A); free(h_B); free(h_C);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║              Tensor Core Basics Exercises                    ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    exercise1_checkTensorCoreSupport();
    exercise2_precisionAnalysis();
    exercise3_basicWMMA();
    
    printf("══════════════════════════════════════════════════════════════\n");
    printf("                    All exercises completed!\n");
    printf("══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o tensor_core_exercises tensor_core_exercises.cu && ./tensor_core_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Check Your GPU

In [ ]:
# Exercise: Determine if your GPU supports Tensor Cores
# and what data types it supports

!nvidia-smi --query-gpu=name,compute_cap --format=csv

### Exercise 2: Precision Analysis

Run a matrix multiply in FP32 vs FP16 and measure:
1. Maximum element-wise error
2. Mean absolute error
3. Speedup ratio

---

## 🎯 Key Takeaways

### Tensor Core Mental Model

```
┌─────────────────────────────────────────────────────────────────┐
│  🧱 LEGO BLOCK ASSEMBLY COMPLETE!                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  What You Learned:                                              │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │ Individual bricks (CUDA Core)  →  Block sections (TC)  │    │
│  │ 1 FMA per cycle               →  64 FMAs per cycle     │    │
│  │ Scalar operations             →  Matrix operations     │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                 │
│  When to Use Tensor Cores:                                      │
│  ✅ Matrix multiplications (AI/ML)                              │
│  ✅ Convolutions (CNNs)                                         │
│  ✅ Attention mechanisms (Transformers)                         │
│  ❌ Irregular memory access patterns                            │
│  ❌ Small matrices (< 16×16)                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Quick Reference

| Concept | Description | LEGO Analogy |
|---------|-------------|--------------|
| Tensor Core | Matrix multiply unit | Block assembly station |
| WMMA | Warp Matrix Multiply API | Assembly instructions |
| Fragment | Matrix piece per thread | Piece of the block |
| FP16/TF32 | Reduced precision | Simplified brick shapes |

### Performance Checklist

- [ ] Matrices padded to multiples of 16
- [ ] Memory aligned for coalesced access
- [ ] Using WMMA-friendly dimensions
- [ ] Precision chosen for task (FP16/TF32/BF16)

### Tomorrow: WMMA Deep Dive
We'll implement optimized matrix multiply using WMMA with shared memory tiling.